In [11]:
# 구글 드라이브에 저장된 파일을 사용하기 위한 코드
from google.colab import drive
drive.mount('/content/drive')

# 저장된 파일의 경로 설정
%cd /content/drive/MyDrive/LG 해커톤 오프라인/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/LG 해커톤 오프라인


In [10]:
# 필요한 라이브러리 불러오기
import random
import math
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
from sklearn.preprocessing import LabelEncoder

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [114]:
train = pd.read_csv("data/train.csv")
train
# Drop non-sales related columns to focus on the sales data
# sales_data = df.drop(columns=['ID', '제품', '대분류', '중분류', '소분류', '브랜드'])

,ID,제품,대분류,중분류,소분류,브랜드,쇼핑몰,2022-01-01,2022-01-02,2022-01-03,...,2023-04-15,2023-04-16,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-22,2023-04-23,2023-04-24
0,SAMPLE_00000,B002-00001-00001,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,S001-00001,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,SAMPLE_00001,B002-00002-00001,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,S001-00001,0,0,0,...,2,0,2,0,2,2,1,0,0,0
2,SAMPLE_00002,B002-00002-00002,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,S001-00001,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,SAMPLE_00003,B002-00002-00003,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,S001-00001,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,SAMPLE_00004,B002-00002-00004,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,S001-00010,0,0,0,...,10,6,4,4,4,0,0,0,0,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28889,SAMPLE_28889,B002-03798-00046,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03798,S001-00001,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28890,SAMPLE_28890,B002-03799-00002,B002-C001-0003,B002-C002-0008,B002-C003-0042,B002-03799,S001-00001,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28891,SAMPLE_28891,B002-03799-00003,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03799,S001-00001,0,0,0,...,2,4,1,0,0,0,4,0,2,0
28892,SAMPLE_28892,B002-03799-00004,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03799,S001-00001,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [115]:
train.groupby('쇼핑몰').count()['ID']

쇼핑몰
S001-00001    11226
S001-00002     3104
S001-00003     3250
S001-00004     6927
S001-00005      519
S001-00006       81
S001-00007      116
S001-00008     1748
S001-00009       79
S001-00010     1844
Name: ID, dtype: int64

In [116]:
train.groupby(['쇼핑몰','중분류']).count()['ID']

쇼핑몰         중분류           
S001-00001  B002-C002-0001    2778
            B002-C002-0002     572
            B002-C002-0003    1069
            B002-C002-0004     746
            B002-C002-0005    1492
                              ... 
S001-00010  B002-C002-0007     242
            B002-C002-0008       7
            B002-C002-0009      83
            B002-C002-0010       1
            B002-C002-0011      15
Name: ID, Length: 103, dtype: int64

In [117]:
sales = pd.read_csv("data/sales.csv")
sales.head()

,ID,제품,대분류,중분류,소분류,브랜드,쇼핑몰,2022-01-01,2022-01-02,2022-01-03,...,2023-04-15,2023-04-16,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-22,2023-04-23,2023-04-24
0,SAMPLE_00000,B002-00001-00001,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,S001-00001,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,SAMPLE_00001,B002-00002-00001,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,S001-00001,0,0,0,...,44800,0,44800,0,44800,44800,22400,0,0,0
2,SAMPLE_00002,B002-00002-00002,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,S001-00001,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,SAMPLE_00003,B002-00002-00003,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,S001-00001,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,SAMPLE_00004,B002-00002-00004,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,S001-00010,0,0,0,...,45740,28360,17320,19200,19200,0,0,0,0,41800


In [118]:
# # onesale : 제품당 판매가격
# # onesale = sales
result = []
i = 0
while i <= 28893: #486: #464->486
    k = 0
    t = list(train.loc[i,:])
    s = list(sales.loc[i,:])
    for j in range(7,486):
        if t[j] == 0:
            pass
        else:
            s[j] //= t[j]
            if k < s[j]:
              k = s[j]
    result.append(k)
    i+=1

train['판매가격'] = result

In [119]:
hh = []
h = 0
while h <28894 :
    ## 중분류 기준으로 '카테고리' 컬럼 추가
    pro = train.iloc[h][3]

    if pro == "B002-C002-0001":
        hh.append('건강기능식품')
    elif pro == "B002-C002-0002":
        hh.append('탈취/방충/살충/제습/방향')
    elif pro == "B002-C002-0003":
        hh.append('주방/청소/세탁세제')
    elif pro == "B002-C002-0004":
        hh.append('욕실용품')
    elif pro == "B002-C002-0005":
        hh.append('제지/위생용품')
    elif pro == "B002-C002-0006":
        hh.append('헤어/바디/스킨/면도')
    elif pro == "B002-C002-0007":
        hh.append('특수헤어용품')
    elif pro == "B002-C002-0008":
        hh.append('유아 생활용품')
    elif pro == "B002-C002-0009":
        hh.append('유아 위생용품')
    elif pro == "B002-C002-0010":
        hh.append('뷰티용품')
    else:
        hh.append('유아식품')
    h += 1

train['카테고리'] = hh
train

,ID,제품,대분류,중분류,소분류,브랜드,쇼핑몰,2022-01-01,2022-01-02,2022-01-03,...,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-22,2023-04-23,2023-04-24,판매가격,카테고리
0,SAMPLE_00000,B002-00001-00001,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,S001-00001,0,0,0,...,0,0,0,0,0,0,0,0,13500,특수헤어용품
1,SAMPLE_00001,B002-00002-00001,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,S001-00001,0,0,0,...,2,0,2,2,1,0,0,0,37800,유아 생활용품
2,SAMPLE_00002,B002-00002-00002,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,S001-00001,0,0,0,...,0,0,0,0,0,0,0,0,12900,유아 생활용품
3,SAMPLE_00003,B002-00002-00003,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,S001-00001,0,0,0,...,0,0,0,0,0,0,0,0,5583,유아 생활용품
4,SAMPLE_00004,B002-00002-00004,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,S001-00010,0,0,0,...,4,4,4,0,0,0,0,8,5225,유아 생활용품
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28889,SAMPLE_28889,B002-03798-00046,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03798,S001-00001,0,0,0,...,0,0,0,0,0,0,0,0,16900,유아 생활용품
28890,SAMPLE_28890,B002-03799-00002,B002-C001-0003,B002-C002-0008,B002-C003-0042,B002-03799,S001-00001,0,0,0,...,0,0,0,0,0,0,0,0,2000,유아 생활용품
28891,SAMPLE_28891,B002-03799-00003,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03799,S001-00001,0,0,0,...,1,0,0,0,4,0,2,0,26400,유아 생활용품
28892,SAMPLE_28892,B002-03799-00004,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03799,S001-00001,0,0,0,...,0,0,0,0,0,0,0,0,14425,유아 생활용품


In [120]:
category_counts = train['카테고리'].value_counts()

# 빈 데이터프레임 생성
result_df = pd.DataFrame()

# 각 카테고리별로 처리
for category in category_counts.index:
    # 카테고리별 데이터 선택
    selected_rows = train[train['카테고리'] == category]

    # '판매 가격' 기준으로 내림차순 정렬
    selected_rows = selected_rows.sort_values(by='판매가격', ascending=False)

    # 길이를 3분할하여 '가격분류' 컬럼 추가
    num_rows = len(selected_rows)
    selected_rows['가격분류'] = pd.cut(range(num_rows), bins=3, labels=['상', '중', '하'])

    # 결과 데이터프레임에 추가
    result_df = pd.concat([result_df, selected_rows])

# 결과 출력
print(result_df)

                 ID                제품             대분류             중분류  \
1928   SAMPLE_01928  B002-00266-00087  B002-C001-0001  B002-C002-0001   
2038   SAMPLE_02038  B002-00266-00142  B002-C001-0001  B002-C002-0001   
1544   SAMPLE_01544  B002-00266-00010  B002-C001-0001  B002-C002-0001   
1593   SAMPLE_01593  B002-00266-00020  B002-C001-0001  B002-C002-0001   
1579   SAMPLE_01579  B002-00266-00017  B002-C001-0001  B002-C002-0001   
...             ...               ...             ...             ...   
27758  SAMPLE_27758  B002-03644-00001  B002-C001-0003  B002-C002-0010   
18248  SAMPLE_18248  B002-02300-00017  B002-C001-0003  B002-C002-0010   
18148  SAMPLE_18148  B002-02283-00030  B002-C001-0003  B002-C002-0010   
10882  SAMPLE_10882  B002-01474-00001  B002-C001-0003  B002-C002-0010   
27754  SAMPLE_27754  B002-03644-00001  B002-C001-0003  B002-C002-0010   

                  소분류         브랜드         쇼핑몰  2022-01-01  2022-01-02  \
1928   B002-C003-0005  B002-00266  S001-00001     

In [121]:
train=result_df.sort_values(by='ID')
train=train.reset_index()
# train.drop('level_0',axis=1,inplace=True)

train.head()

,index,ID,제품,대분류,중분류,소분류,브랜드,쇼핑몰,2022-01-01,2022-01-02,...,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-22,2023-04-23,2023-04-24,판매가격,카테고리,가격분류
0,0,SAMPLE_00000,B002-00001-00001,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,S001-00001,0,0,...,0,0,0,0,0,0,0,13500,특수헤어용품,중
1,1,SAMPLE_00001,B002-00002-00001,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,S001-00001,0,0,...,0,2,2,1,0,0,0,37800,유아 생활용품,상
2,2,SAMPLE_00002,B002-00002-00002,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,S001-00001,0,0,...,0,0,0,0,0,0,0,12900,유아 생활용품,중
3,3,SAMPLE_00003,B002-00002-00003,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,S001-00001,0,0,...,0,0,0,0,0,0,0,5583,유아 생활용품,하
4,4,SAMPLE_00004,B002-00002-00004,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,S001-00010,0,0,...,4,4,0,0,0,0,8,5225,유아 생활용품,하


In [122]:
train['카테고리'].value_counts()

건강기능식품            6650
제지/위생용품           5206
특수헤어용품            4203
주방/청소/세탁세제        3565
헤어/바디/스킨/면도       2734
욕실용품              1993
유아 위생용품           1798
탈취/방충/살충/제습/방향    1330
유아식품               915
유아 생활용품            460
뷰티용품                40
Name: 카테고리, dtype: int64

In [123]:
train.groupby(['카테고리','가격분류']).agg({'판매가격':'max'})

판매가격
카테고리           가격분류        
건강기능식품         상     819000
               중      38400
               하      19800
뷰티용품           상      33000
               중      16000
               하       9900
욕실용품           상     505000
               중       7300
               하       2550
유아 생활용품        상     389000
               중      22900
               하       9900
유아 위생용품        상     126900
               중      24100
               하      12900
유아식품           상      91900
               중       4160
               하       1900
제지/위생용품        상     502300
               중      13500
               하       2875
주방/청소/세탁세제     상     854118
               중      12880
               하       6225
탈취/방충/살충/제습/방향 상     159000
               중      10500
               하       4640
특수헤어용품         상     340000
               중      18000
               하       9500
헤어/바디/스킨/면도    상     658700
               중      22900
               하       8100

In [124]:
train.drop(['중분류'],axis=1,inplace=True)
train.to_csv('train_off1.csv')